In [1]:
# Mounting a new session
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# get the data from the zip file
!unzip -uq "/content/drive/My Drive/DATA/auth_day_9_labelled.zip" -d "/content/drive/My Drive/DATA"

In [0]:
import numpy as np
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import tensorflow as tf
# if tf.__version__ < "2.0":
#   !pip install --upgrade tensorflow

from keras.models import Model, load_model
from keras.layers import LSTM, Dense, Dense, Dropout, GaussianNoise
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.layers import Input
from keras import regularizers
from sklearn.preprocessing import LabelEncoder
from keras.layers.normalization import BatchNormalization
from keras.optimizers import adagrad, RMSprop, Adam
from scipy.stats import lognorm
import datetime

from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc,
                              roc_curve, recall_score, classification_report, f1_score,
                              precision_recall_fscore_support)

%matplotlib inline

In [0]:
class LSTM_autoencoder:
    """
    This function implements lstm autoencoder. It trains the model on normal log-lines and
    evaluates the model in the final phase
    
    """
    
    
    def __init__(self):
        pass
    
        
    def split_user_domain(self, dataset):
        """
        This function is for data cleaning - removing missing values
        
        """
        dataset['source_user'] = dataset['source_user@domain'].str.split('@').str[0]
        dataset['source_user'] = np.where(dataset['source_user'].str.contains('$'), dataset['source_user'].str.split('$').str[0],
                                    dataset['source_user'])
        dataset['source_domain'] = dataset['source_user@domain'].str.split('@').str[1]

        dataset['destination_user'] = dataset['destination_user@domain'].str.split('@').str[0]
        dataset['destination_user'] = np.where(dataset['destination_user'].str.contains('$'), 
                                               dataset['destination_user'].str.split('$').str[0],
                                         dataset['destination_user'])
        dataset['destination_domain'] = dataset['destination_user@domain'].str.split('@').str[1]
        
        self.dataset = dataset
    
    def drop_columns(self):
        """ This function drops the irrelevant columns. """
               
        self.dataset = self.dataset.drop(['time', 'source_user@domain', 'destination_user@domain'], axis = 1)
        
#         return self.dataset

    def transform_data(self):
        """ This function encodes each column based on the values. """
        
        from sklearn.preprocessing import LabelEncoder
        lb = LabelEncoder()
        for cols in self.dataset.columns:
            self.dataset[cols] = lb.fit_transform(self.dataset[cols])

    def normalize(self):
        """ This function normalizes each column based on the range of the column. """
        from sklearn.preprocessing import StandardScaler
        from sklearn.preprocessing import MinMaxScaler

        for cols in self.dataset.columns:
            self.dataset[cols] = MinMaxScaler(feature_range=(0,1)).fit_transform(self.dataset[cols].values.reshape(-1,1))
        
    def sep_x_y(self):
        """ this function separates the """
        features = list()
        for cols in self.dataset.columns:
            if cols != 'Malignant/Benign':
                features.append(cols)
        self.x = self.dataset[features]
        self.y = self.dataset['Malignant/Benign']
        
#         return self.x,self.y

    def df_to_array(self):
        self.x = self.x.values
        
#         return self.x
    
    def shape_to_3dim(self):
        """ This function changes the shape of the array for the LSTM -autoencoder.
            This has to be done for test set as well.
            Only for features (i.e x-values)
        """
        self.x = self.x.reshape(self.x.shape[0], 1, self.x.shape[1])
#         return self.x
    def pred(self):
        self.predictions = self.autoencoder.predict(self.x)
#         return self.predictions
    
    
    def fit(self, nb_epoch = 100, batch_size = 512):
            ## Creating the Model
            repeat_vector = self.x.shape[1]
            output_dim = self.x.shape[2] # number of features
            learning_rate = 1e-6
            max_len = 120
            num_chars = 128
            layers = 128
            autoencoder = Sequential()
            # Encoder
            # autoencoder.add(LSTM(layers, return_sequences=True, input_shape=(max_len, num_chars),
            #                      dropout = 0.5))
            autoencoder.add(LSTM(layers, return_sequences=True))
            autoencoder.add(GaussianNoise(0.8))
            autoencoder.add(BatchNormalization())
            autoencoder.add(Dropout(0.5))
            # autoencoder.add(RepeatVector(repeat_vector))

            autoencoder.add(Dense(128, activation='relu'))
            autoencoder.add(BatchNormalization())
            autoencoder.add(Dense(output_dim, activation='softmax'))
            # autoencoder.add(TimeDistributed(Dense(output_dim, activation='softmax')))

            # autoencoder.add(LSTM(256, activation='relu', activity_regularizer=regularizers.l1(learning_rate),return_sequences=True))
            # autoencoder.add(LSTM(128, activation='sigmoid', activity_regularizer=regularizers.l2(learning_rate),return_sequences=False))
            # autoencoder.add(Dense(64, activation = 'softmax'))
            # autoencoder.add(Dropout(0.5))
            # autoencoder.add(GaussianNoise(0.8))
            # autoencoder.add(BatchNormalization())
            # autoencoder.add(Dense(32, activation = 'relu'))
            # autoencoder.add(Dropout(0.5))
            # autoencoder.add(Dense(16, activation='sigmoid'))
            # autoencoder.add(RepeatVector(repeat_vector))

            # # Decoder
            # autoencoder.add(Dense(16, activation='sigmoid'))
            # autoencoder.add(Dense(32, activation='relu'))
            # autoencoder.add(Dense(64, activation= 'sigmoid'))
            # autoencoder.add(LSTM(128, activation='sigmoid', activity_regularizer=regularizers.l2(learning_rate),return_sequences=True))
            # autoencoder.add(LSTM(256, activation='relu', activity_regularizer=regularizers.l1(learning_rate),return_sequences=True))
            # autoencoder.add(TimeDistributed(Dense(output_dim)))

            # Model Compiler
            # opt = adagrad(lr = 1e-3, decay = 1e-5)
            # opt = RMSprop(lr = 0.001, epsilon = 1e-06)
            opt = Adam(lr = 0.001, epsilon = 1e-06)

            autoencoder.compile(metrics=['accuracy'],
                                loss='mean_squared_error',
                                optimizer= opt)
            autoencoder.fit(self.x, self.x, epochs=nb_epoch, 
                                      batch_size = batch_size, shuffle = True ,verbose = 0)
            self.autoencoder = autoencoder

    


    def prediction_results(self):
            
            self.y = self.y.reset_index()['Malignant/Benign']
            self.x = self.x.reshape(self.x.shape[0], self.x.shape[2])
            self.predictions = self.predictions.reshape(self.predictions.shape[0], self.predictions.shape[2])

            mse = np.mean(np.power(self.x - self.predictions, 2), axis=1)

            error_df = pd.DataFrame({'reconstruction_error': mse,
                                'true_class': self.y})
            
            fpr, tpr, thresholds = roc_curve(error_df.true_class, error_df.reconstruction_error)
            roc_auc = auc(fpr, tpr)
            
            threshold = min(thresholds)
            LABELS = ["Normal", "MALIGNANT"]
            y_pred = [1 if e > threshold else 0 for e in error_df.reconstruction_error.values]
            conf_matrix = confusion_matrix(error_df.true_class, y_pred)
            
            ##Performance check of the model

            TP = FN = FP = TN = 0

            for j in range(len(self.y)):
                if self.y[j]== 1 and y_pred[j] == 1:
                    TP = TP+1
                elif self.y[j]== 1 and y_pred[j] == 0:
                    FN = FN+1
                elif self.y[j]== 0 and y_pred[j] == 1:
                    FP = FP+1
                elif self.y[j] == 0 and y_pred[j] == 0:
                    TN = TN +1
            # print (TP,  FN,  FP,  TN)

            TPR = TP/(TP+FN)
            FPR = FP/(FP+TN)
            TNR = TN/(TN+FP)
            FNR = FN/(FN+TP)


            accuracy = (TP+TN)/(TP+FN+FP+TN)
            sensitivity = TP/(TP+FN)
            specificity = TN/(TN+FP)
            precision = TP/(TP+FP) 

            print ("TPR: ", np.round(TPR, 4), 
                  "; FPR: ", np.round(FPR, 4),
                  "; TNR: ", np.round(TNR, 4),
                  "; FNR: ", np.round(FNR, 4))

            print ("Accuracy: ", np.round(accuracy, 4), 
                  "; Sensitivity: ", np.round(sensitivity, 4),
                  "; Specificity: ", np.round(specificity, 4),
                  "; Precision: ", np.round(precision, 4),
                  "; F1-score: " , np.round(2/((1/precision) + (1/sensitivity)),4))
            
#             return self.error_df


    def create_subsets(self,test_set):

            n1, n2, n3 = 500,300,200
            q1, q2, q3 = 200,200,200
            normal = test_set[test_set['Malignant/Benign'] == 0]
            abnormal = test_set[test_set['Malignant/Benign'] == 1]
            t1 =  normal[:n1]
            t2 =  normal[n1:n1+n2]
            t3 =  normal[n2:n2+n3]

            t1 = t1.append(abnormal)
            t2 = t2.append(abnormal)
            t3 = t3.append(abnormal)
            
#             self.t1, self.t2, self.t3 = t1,t2,t3

            return t1, t2, t3





In [45]:
df_9 = pd.read_csv("/content/drive/My Drive/DATA/auth_day_9_labelled.txt", header = None)
test_1 = pd.read_csv("/content/drive/My Drive/DATA/Test_day_9.txt")
df_9.columns = ["time","source_user@domain","destination_user@domain","source_computer","destination_computer",
              "authentication_type","logon_type","authentication_orientation","success/failure", "Malignant/Benign"]

lstm = LSTM_autoencoder()
## Using the training data only
# Training
lstm.split_user_domain(df_9[:1000]) ## change 1000 to more rows
lstm.drop_columns()
lstm.transform_data()
lstm.normalize()
lstm.sep_x_y()
lstm.df_to_array()
lstm.shape_to_3dim()
lstm.fit()

## Testing
## Now use the test set - first with 10,200 mixed observations
t1,t2,t3 = lstm.create_subsets(test_1[test_1.columns[1:]])
t = [test_1[test_1.columns[1:]], t1,t2,t3,]
for test_set in t:
    lstm.split_user_domain(test_set)
    lstm.drop_columns()
    lstm.transform_data()
    lstm.normalize()
    lstm.sep_x_y()
    lstm.df_to_array()
    lstm.shape_to_3dim()
    lstm.pred()
    lstm.prediction_results()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/

TPR:  1.0 ; FPR:  0.9999 ; TNR:  0.0001 ; FNR:  0.0
Accuracy:  0.0197 ; Sensitivity:  1.0 ; Specificity:  0.0001 ; Precision:  0.0196 ; F1-score:  0.0385
TPR:  1.0 ; FPR:  0.998 ; TNR:  0.002 ; FNR:  0.0
Accuracy:  0.2871 ; Sensitivity:  1.0 ; Specificity:  0.002 ; Precision:  0.2861 ; F1-score:  0.4449
TPR:  1.0 ; FPR:  0.9967 ; TNR:  0.0033 ; FNR:  0.0
Accuracy:  0.402 ; Sensitivity:  1.0 ; Specificity:  0.0033 ; Precision:  0.4008 ; F1-score:  0.5722
TPR:  1.0 ; FPR:  0.995 ; TNR:  0.005 ; FNR:  0.0
Accuracy:  0.5025 ; Sensitivity:  1.0 ; Specificity:  0.005 ; Precision:  0.5013 ; F1-score:  0.6678


In [41]:
lstm.autoencoder.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_19 (LSTM)               (None, 1, 128)            71168     
_________________________________________________________________
gaussian_noise_9 (GaussianNo (None, 1, 128)            0         
_________________________________________________________________
batch_normalization_11 (Batc (None, 1, 128)            512       
_________________________________________________________________
dropout_12 (Dropout)         (None, 1, 128)            0         
_________________________________________________________________
dense_27 (Dense)             (None, 1, 128)            16512     
_________________________________________________________________
batch_normalization_12 (Batc (None, 1, 128)            512       
_________________________________________________________________
dense_28 (Dense)             (None, 1, 10)           

In [48]:
lstm.autoencoder.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (None, 1, 128)            71168     
_________________________________________________________________
gaussian_noise_10 (GaussianN (None, 1, 128)            0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 1, 128)            512       
_________________________________________________________________
dropout_13 (Dropout)         (None, 1, 128)            0         
_________________________________________________________________
dense_29 (Dense)             (None, 1, 128)            16512     
_________________________________________________________________
batch_normalization_14 (Batc (None, 1, 128)            512       
_________________________________________________________________
dense_30 (Dense)             (None, 1, 10)           